In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
from bs4 import BeautifulSoup as bs
import re
import pickle

In [2]:
# Found using Postman and intercepting traffics while visiting topuniversities.com
top_universities_url = 'https://www.topuniversities.com'
top_universities_ranking_url = top_universities_url + '/sites/default/files/qs-rankings-data/357051.txt'

the_universities_url = 'https://www.timeshighereducation.com'
the_universities_ranking_url = the_universities_url + '/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'

## Top Universities

In [28]:
# Remove any characters that is not part of a floating number
def match_number(text):
    ''' [-+]?[0-9]*\.?[0-9]+ '''
    return float(re.sub('[^-+0-9.]', '', text))

# Extract a float number from an html document using the selector
def extract_number(html, selector):
    number_tag = html.select_one(selector)
    if number_tag is None:
        return float('NaN')
    else:
        return match_number(number_tag.get_text())

def fetch_top_university_infos(university):
    # Fetch the Webpage of the university
    university_request = rq.get(top_universities_url + university['url'])
    university_html = bs(university_request.text, 'html.parser')
    university_data = dict()
    
    # Scrap data from the JSON and the Webpage
    university_data['name'] = university['title']
    university_data['rank'] = int(match_number(university['rank_display']))
    university_data['region'] = university['region']
    university_data['country'] = university['country']
    university_data['faculty_total'] = extract_number(university_html, 'div.faculty.total div.number')
    university_data['faculty_inter'] = extract_number(university_html, 'div.faculty.inter div.number')
    university_data['faculty_local'] = university_data['faculty_total'] - university_data['faculty_inter']
    university_data['student_total'] = extract_number(university_html, 'div.student.total div.number')
    university_data['student_inter'] = extract_number(university_html, 'div.inter.total div.number')
    university_data['student_local'] = university_data['student_total'] - university_data['student_inter']
    
    return university_data

try:
    # Try to load the serialized data
    top_universities_file = open('top_universities.pickle', 'rb')
    top_universities = pickle.load(top_universities_file)
except:
    # If loading from the serialized data failed, scrap from the web!
    top_universities_json = (rq.get(top_universities_ranking_url).json())['data']
    top_universities = pd.DataFrame(list(map(fetch_top_university_infos, top_universities_json[0:200])))
    
    top_universities.region = top_universities.region.astype('category')
    top_universities.country = top_universities.country.astype('category')
    
    # Serialize the result for future runs
    top_universities_file = open('top_universities.pickle', 'wb')
    pickle.dump(top_universities, top_universities_file)
    
# Create the mapping country => region for the Times scrapper
country_regions = dict()
for country_region_pair, universities in top_universities.groupby(['country', 'region']):
    country_regions[country_region_pair[0]] = country_region_pair[1]
    
top_universities

,country,faculty_inter,faculty_local,faculty_total,name,region,student_inter,student_local,student_total,top_rank
0,United States,1679.0,1303.0,2982.0,Massachusetts Institute of Technology (MIT),North America,3717.0,7350.0,11067.0,1
1,United States,2042.0,2243.0,4285.0,Stanford University,North America,3611.0,12267.0,15878.0,2
2,United States,1311.0,3039.0,4350.0,Harvard University,North America,5266.0,17163.0,22429.0,3
3,United States,350.0,603.0,953.0,California Institute of Technology (Caltech),North America,647.0,1608.0,2255.0,4
4,United Kingdom,2278.0,3212.0,5490.0,University of Cambridge,Europe,6699.0,12071.0,18770.0,5
5,United Kingdom,2964.0,3786.0,6750.0,University of Oxford,Europe,7353.0,12367.0,19720.0,6
6,United Kingdom,2554.0,3791.0,6345.0,UCL (University College London),Europe,14854.0,16226.0,31080.0,7
7,United Kingdom,2071.0,1859.0,3930.0,Imperial College London,Europe,8746.0,7344.0,16090.0,8
8,United States,635.0,1814.0,2449.0,University of Chicago,North America,3379.0,10178.0,13557.0,9
9,Switzerland,1886.0,591.0,2477.0,ETH Zurich - Swiss Federal Institute of Techno...,Europe,7563.0,12252.0,19815.0,10


In [4]:
# Sort the universities by faculty/student ratio and international students/local students ratio
def ratio_stats(universities):
    universities['faculty_student_ratio'] = universities.faculty_total / universities.student_total
    universities['inter_student_ratio'] = universities.student_inter / universities.student_total
    
    top_by_faculty = (universities.sort_values('faculty_student_ratio', ascending = False))[:20]
    top_by_inter = (universities.sort_values('inter_student_ratio', ascending = False))[:20]
    
    return (top_by_faculty, top_by_inter)

In [5]:
# Top universities, using ratios
top_universities_by_faculty, top_universities_by_inter = ratio_stats(top_universities)

# Top universities by country, using ratios
top_country = top_universities.groupby('country').sum()
top_country_by_faculty, top_country_by_inter = ratio_stats(top_country)

# Top universities by region, using ratios
top_regions = top_universities.groupby('region').sum()
top_regions_by_faculty, top_regions_by_inter = ratio_stats(top_regions)

## Times Higher Education

In [29]:
import math

def fetch_the_university_infos(university):
    # No need to fetch more data
    university_data = dict()
    
    student_faculty_ratio = match_number(university['stats_student_staff_ratio'])
    inter_student_ratio = match_number(university['stats_pc_intl_students']) / 100.0
    
    # Scrap data from the JSON, some computation required
    university_data['name'] = university['name']
    university_data['rank'] = int(match_number(university['rank']))
    university_data['region'] = country_regions[university['location']]
    university_data['country'] = university['location']
    university_data['student_total'] = int(match_number(university['stats_number_students']))
    university_data['student_inter'] = int(university_data['student_total'] * inter_student_ratio)
    university_data['student_local'] = university_data['student_total'] - university_data['student_inter']
    university_data['faculty_total'] = int(university_data['student_total'] / student_faculty_ratio)
    university_data['faculty_inter'] = float('NaN')
    university_data['faculty_local'] = float('NaN')
    
    return university_data
    
try:
    # Try to load the serialized data
    the_universities_file = open('the_universities.pickle', 'rb')
    the_universities = pickle.load(the_universities_file)
except:
    # If loading from the serialized data failed, scrap from the web!
    the_universities_json = (rq.get(the_universities_ranking_url).json())['data']
    the_universities = pd.DataFrame(list(map(fetch_the_university_infos, the_universities_json[0:200])))
        
    the_universities.region = the_universities.region.astype('category')
    the_universities.country = the_universities.country.astype('category')
    
    # Serialize the result for future runs
    the_universities_file = open('the_universities.pickle', 'wb')
    pickle.dump(the_universities, the_universities_file)

In [7]:
# Top universities, using ratios
the_universities_by_faculty, the_universities_by_inter = ratio_stats(the_universities)

# Top universities by country, using ratios
the_country = the_universities.groupby('country').sum()
the_country_by_faculty, the_country_by_inter = ratio_stats(the_country)

# Top universities by region, using ratios
the_regions = the_universities.groupby('region').sum()
the_regions_by_faculty, the_regions_by_inter = ratio_stats(the_regions)

# Merging datasets

In [23]:
from difflib import SequenceMatcher
from collections import defaultdict
from operator import itemgetter

def proximity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# We will compute the proximity of the name between all universities of the rankings (cross product)
proximities = dict()
for top_index, top_uni in top_universities.iterrows():
    for the_index, the_uni in the_universities.iterrows():
        # Don't need to work on universities in different countries
        if top_uni['country'] != the_uni['country']:
            continue
        
        prox = proximity(top_uni['name'], the_uni['name'])
        
        # Only keep good proximity to reduce work in the next phase
        if prox >= .5:
            proximities[(top_index, the_index)] = prox

In [33]:
# Prepare universities to be matched
top_universities['matched'] = False
the_universities['matched'] = False
the_universities['match_name'] = None
the_universities['prox'] = 0.0

# Sort match by quality, we want the best matches to be processed first in case of conflicts
best_matches = sorted(proximities.items(), key = itemgetter(1), reverse = True)

for indexes, proximity in best_matches:
    # Get a view on both unis
    top_uni, the_uni = top_universities.loc[indexes[0]], the_universities.loc[indexes[1]]
    
    # Do the match only if both are still free
    if not top_uni['matched'] and not the_uni['matched']:
        # Mark the universities as matched
        top_universities.loc[indexes[0],'matched'] = True
        the_universities.loc[indexes[1],'matched'] = True
        # Give the same name for the merge
        the_universities.loc[indexes[1],'prox'] = proximity
        the_universities.loc[indexes[1],'merge_name'] = top_uni['name']

# Use a join to merge the datasets using the name
merged_universities = top_universities.merge(the_universities, how = 'inner', left_on = 'name', right_on = 'merge_name')
(merged_universities.sort_values('prox'))

,country_x,faculty_inter_x,faculty_local_x,faculty_total_x,name_x,region_x,student_inter_x,student_local_x,student_total_x,top_rank,...,name_y,region_y,student_inter_y,student_local_y,student_total_y,the_rank,matched_y,match_name,prox,merge_name
157,Italy,1.0,118.0,119.0,Scuola Superiore Sant'Anna Pisa di Studi Unive...,Europe,135.0,654.0,789.0,192,...,Scuola Superiore Sant’Anna,Europe,96,594,690,155,True,None,0.500000,Scuola Superiore Sant'Anna Pisa di Studi Unive...
151,Belgium,515.0,1277.0,1792.0,Vrije Universiteit Brussel (VUB),Europe,1853.0,7431.0,9284.0,182,...,Université Libre de Bruxelles,Europe,8870,16473,25343,175,True,None,0.524590,Vrije Universiteit Brussel (VUB)
113,France,253.0,2020.0,2273.0,Université Pierre et Marie Curie (UPMC),Europe,4213.0,20092.0,24305.0,131,...,Pierre and Marie Curie University,Europe,5023,20094,25117,123,True,None,0.583333,Université Pierre et Marie Curie (UPMC)
115,Sweden,336.0,741.0,1077.0,Chalmers University of Technology,Europe,1465.0,7792.0,9257.0,133,...,University of Gothenburg,Europe,2389,15989,18378,198,True,None,0.596491,Chalmers University of Technology
145,Germany,413.0,1553.0,1966.0,Albert-Ludwigs-Universitaet Freiburg,Europe,3897.0,19317.0,23214.0,171,...,University of Freiburg,Europe,3965,16906,20871,82,True,None,0.620690,Albert-Ludwigs-Universitaet Freiburg
141,Germany,796.0,3048.0,3844.0,Eberhard Karls Universität Tübingen,Europe,3860.0,23506.0,27366.0,164,...,University of Tübingen,Europe,3619,24225,27844,94,True,None,0.631579,Eberhard Karls Universität Tübingen
125,Germany,377.0,2185.0,2562.0,Technische Universität Berlin (TU Berlin),Europe,4494.0,17682.0,22176.0,144,...,Technical University of Berlin,Europe,4478,17912,22390,92,True,None,0.676056,Technische Universität Berlin (TU Berlin)
133,Spain,108.0,3755.0,3863.0,Universitat de Barcelona,Europe,6134.0,39868.0,46002.0,156,...,Autonomous University of Barcelona,Europe,5169,27140,32309,147,True,None,0.689655,Universitat de Barcelona
154,United Kingdom,422.0,858.0,1280.0,University of Reading,Europe,4077.0,9253.0,13330.0,188,...,University of Sussex,Europe,4705,7679,12384,147,True,None,0.731707,University of Reading
137,United Kingdom,440.0,725.0,1165.0,University of Bath,Europe,4213.0,10157.0,14370.0,160,...,University of East Anglia,Europe,4056,9465,13521,188,True,None,0.744186,University of Bath
